In [103]:
import pandas as pd
import pymysql 
from sqlalchemy import create_engine
from config import db_username, db_password, db_name, db_port_mysql, db_host

In [104]:
df = pd.read_csv("universities-clean.csv")
df

,Rank,Institution,City,Country,Continent,Founded,Affiliation,AttendanceMode,Enrollment,Link
0,1,Indira Gandhi National Open University,New Delhi,India,Asia,1985,Public,Distance/In-Person,7140000,https://en.wikipedia.org/wiki/Indira_Gandhi_Na...
1,2,"National University, Bangladesh",Gazipur,Bangladesh,Asia,1992,Public,In-Person,3425832,https://en.wikipedia.org/wiki/National_Univers...
2,3,Anadolu University,Eskişehir,Turkey,Asia,1958,Public,Distance/In-Person,1974343,https://en.wikipedia.org/wiki/Anadolu_University
3,4,California Community Colleges,California,United States,North America,1967,Public,In-Person,1800000,https://en.wikipedia.org/wiki/California_Commu...
4,5,Islamic Azad University,Iran,Iran,Asia,1982,Private,In-Person,1778000,https://en.wikipedia.org/wiki/Islamic_Azad_Uni...
...,...,...,...,...,...,...,...,...,...,...
85,85,Aix-Marseille University,Marseille,France,Europe,1409,Public,In-Person,80000,https://en.wikipedia.org/wiki/Aix-Marseille_Un...
86,86,University of Hagen,Hagen,Germany,Europe,1974,Public,Distance,69982,https://en.wikipedia.org/wiki/University_of_Hagen
87,87,Université de Montréal,Canada,Canada,North America,1878,Public,Distance/In-Person,67109,https://en.wikipedia.org/wiki/Universit%C3%A9_...
88,88,University of the Philippines,Philippines,Philippines,Asia,1908,Public,Distance/In-Person,64144,https://en.wikipedia.org/wiki/University_of_th...


In [105]:
conn = pymysql.Connect(
    host=db_host,
    port=db_port_mysql,
    password=db_password,
    user=db_username,
    database=db_name
)

In [106]:
cursor = conn.cursor()

In [107]:
cursor.execute("SHOW TABLES;")

1

In [108]:
cursor.fetchall()

(('university',),)

In [109]:
create_query_file = open('./sql/mysql_create_table.sql')
create_query = create_query_file.read()
create_query

'CREATE TABLE university (\n    id BIGINT AUTO_INCREMENT NOT NULL,\n    `rank` BIGINT NOT NULL,\n    `name` VARCHAR(255) NOT NULL,\n    city VARCHAR(255) NOT NULL,\n    country VARCHAR(255) NOT NULL,\n    continent VARCHAR(255) NOT NULL,\n    founded INT NOT NULL,\n    affiliation VARCHAR(255) NOT NULL,\n    attendancemode VARCHAR(255) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    `link` VARCHAR(255) NOT NULL,\n    UNIQUE KEY unique_combination (country, `name`),\n    PRIMARY KEY (id)\n);\n'

In [110]:
cursor.execute(create_query)

OperationalError: (1050, "Table 'university' already exists")

In [69]:
data = list(df.itertuples(index=None, name=None))

In [62]:
engine = create_engine("mysql+pymysql://", creator=lambda: conn) #specify existing connection into mysql file

In [123]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,rank,name,city,country,continent,founded,affiliation,attendancemode,enrollment,link


In [117]:
merge_query_file = open('./sql/mysql_upsert.sql')
merge_query = merge_query_file.read()
merge_query

'INSERT INTO university (\n    `rank`, `name`, city, country, continent, founded, \n    affiliation, attendancemode, enrollment, `link`)\nVALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)\nON DUPLICATE KEY UPDATE\n    `rank` = VALUES(`rank`),\n    `name` = VALUES(`name`),\n    city = VALUES(city),\n    country = VALUES(country),\n    continent = VALUES(continent),\n    founded = VALUES(founded),\n    affiliation = VALUES(affiliation),\n    attendancemode = VALUES(attendancemode),\n    enrollment = VALUES(enrollment),\n    `link` = VALUES(`link`);\n'

In [118]:
data

[(1,
  'Indira Gandhi National Open University',
  'New Delhi',
  'India',
  'Asia',
  1985,
  'Public',
  'Distance/In-Person',
  7140000,
  'https://en.wikipedia.org/wiki/Indira_Gandhi_National_Open_University'),
 (2,
  'National University, Bangladesh',
  'Gazipur',
  'Bangladesh',
  'Asia',
  1992,
  'Public',
  'In-Person',
  3425832,
  'https://en.wikipedia.org/wiki/National_University,_Bangladesh'),
 (3,
  'Anadolu University',
  'Eskişehir',
  'Turkey',
  'Asia',
  1958,
  'Public',
  'Distance/In-Person',
  1974343,
  'https://en.wikipedia.org/wiki/Anadolu_University'),
 (4,
  'California Community Colleges',
  'California',
  'United States',
  'North America',
  1967,
  'Public',
  'In-Person',
  1800000,
  'https://en.wikipedia.org/wiki/California_Community_Colleges'),
 (5,
  'Islamic Azad University',
  'Iran',
  'Iran',
  'Asia',
  1982,
  'Private',
  'In-Person',
  1778000,
  'https://en.wikipedia.org/wiki/Islamic_Azad_University'),
 (6,
  'Allama Iqbal Open University'

In [119]:
cursor.executemany(merge_query, data)

90

In [124]:
conn.commit()

In [125]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,rank,name,city,country,continent,founded,affiliation,attendancemode,enrollment,link
0,1,1,Indira Gandhi National Open University,New Delhi,India,Asia,1985,Public,Distance/In-Person,7140000,https://en.wikipedia.org/wiki/Indira_Gandhi_Na...
1,2,2,"National University, Bangladesh",Gazipur,Bangladesh,Asia,1992,Public,In-Person,3425832,https://en.wikipedia.org/wiki/National_Univers...
2,3,3,Anadolu University,Eskişehir,Turkey,Asia,1958,Public,Distance/In-Person,1974343,https://en.wikipedia.org/wiki/Anadolu_University
3,4,4,California Community Colleges,California,United States,North America,1967,Public,In-Person,1800000,https://en.wikipedia.org/wiki/California_Commu...
4,5,5,Islamic Azad University,Iran,Iran,Asia,1982,Private,In-Person,1778000,https://en.wikipedia.org/wiki/Islamic_Azad_Uni...
...,...,...,...,...,...,...,...,...,...,...,...
85,86,85,Aix-Marseille University,Marseille,France,Europe,1409,Public,In-Person,80000,https://en.wikipedia.org/wiki/Aix-Marseille_Un...
86,87,86,University of Hagen,Hagen,Germany,Europe,1974,Public,Distance,69982,https://en.wikipedia.org/wiki/University_of_Hagen
87,88,87,Université de Montréal,Canada,Canada,North America,1878,Public,Distance/In-Person,67109,https://en.wikipedia.org/wiki/Universit%C3%A9_...
88,89,88,University of the Philippines,Philippines,Philippines,Asia,1908,Public,Distance/In-Person,64144,https://en.wikipedia.org/wiki/University_of_th...
